# <center> Solving the Two-Language Problem with Julia </center>

### <center> Paul Stey, Ph.D. </center>
### <center> Brown University</center>
### <center> Center for Computation \& Visualization</center>

### <center> 2019-05-30 </center>

## Outline 

1. The Two-Language Problem
2. Speed & Parallelism
3. Type System and Multiple Dispatch
4. Profiling and Introspection
5. Integrations with other Languages
6. Tooling and Package Ecosystem

## Two-Language Problem
  - Dynamic languages (e.g., Python, R, Matlab) tend to be slow
  - Lower-level, compiled languages (e.g., C, C++, Fortran) are very fast, but are more time-consuming to write and debug 

 ## The Standard Approach 
   - Write core algorithms in lower-level language
   - Then wrap that in higher-level language using some interface language/package (e.g., Cython, Rcpp)

## What's the Problem?
  1. Higher barrier to entry for package authors
  2. Package authors must know (at least) two languages
  3. Creates a sharp divide between package authors and package users

# <center> Meet Julia! </center>




## Julia is:
1. Dynamic technical computing lanugage
2. Multi-paradigm (functional, imperative, "OO"-ish)
3. Just-in-time compiled using LLVM
4. Under active development (current stable version: 1.1.1)
5. Fast!
6. Fun!

## History of Julia

* Started in 2009 as PhD thesis of Jeff Bezanson at MIT
* Jeff has been collaborating with Stefan Karpinski and Viral Shah
* First official release in 2012
* Influenced by: C, Python, R, Ruby, Lisp
* Specifically designed to be as fast as C and as expressive as Python or Ruby

## Julia Basics
- Similar to Python and R in that:
  * Very flexible
  * Has elements of imperative, functional, and object-oriented programming
  * Functions are first-class citizens 
    + (e.g., pass as arguments, return from other functions)
 


## Julia Basics (cont.)
- Differs from Python and R in that:
  * Compiled, not interpretted
  * Not object-oriented in the classical sense
  * No classes with private data and private methods
  * No concept of inheritence
  * Designed with parallelism in mind
  * Multiple dispatch
  * Excellent for generic programming
  * Metaprogramming

## Base Language Data Types

1. Primitives
  * Any numeric type you can imagine: `Int64`, `Float64`, `BigInt`, `Complex`, `Irrational`, `Rational`
  * Many abstract types: `Any`, `Real`, `Number`, `Integer`
  * `String` and `Chars`
2. Container Types
  * `Array` (vectors, matrices, _N_-dimensional arrays)
  * `Set`
  * `Dict`
  * `Tuple` and `NamedTuple`
3. Composite Types
  * `struct`

In [85]:
# Simple function that computes Fibonacci numbers

function fib(n)
    nums = ones(Int, n)
    for i = 3:n
        nums[i] = nums[i - 1] + nums[i - 2]
    end 
    return nums[n]
end     

fib (generic function with 1 method)

In [86]:
fib(30)

832040

# <center> Julia is Fast! </center>

In [88]:
function qsort(a, lo, hi)
    i = lo
    j = hi
    while i < hi
        pivot = a[div(lo + hi, 2)]
        while i <= j
            while a[i] < pivot
                i += 1
            end
            while a[j] > pivot
                j -= 1
            end
            if i <= j
                a[i], a[j] = a[j], a[i]
                i += 1
                j -= 1
            end 
        end
        if lo < j
            qsort(a, lo, j)
        end
        lo = i
        j = hi
    end 
    return a
end 

qsort (generic function with 1 method)

### <center>Python `quicksort` example</center>

In [90]:
# quick sort in Julia

n = 1_000_000
x = rand(n)

@time x2 = qsort(x, 1, n);

  0.091415 seconds (4 allocations: 160 bytes)


## Understanding Julia's Speed

1. Type system makes inferring concrete types easy
2. Julia aggressively specializes run-time types
3. LLVM generates fast native code
4. Very smart people working hard!

# <center> Parrallelism in Julia </center>

## Parallel Programming in Julia

1. Can use multi-processing or multi-threading
2. Multi-threading works using "green threads"
3. Excellent support for SIMD operations 
4. `SharedArray` data type in base language
3. `DistributedArray` data structure allows for Hadoop-like distributed computing

In [91]:
# Parallel processing with @distributed 
using Distributed 

addprocs(5)                 

function monte_carlo_pi(n_iter)
    num_inside = @distributed (+) for i = 1:n_iter
        Int(rand()^2 + rand()^2 ≤ 1)
    end
    res = num_inside/(n_iter/4)
    return res
end 

monte_carlo_pi (generic function with 1 method)

In [94]:
@time monte_carlo_pi(100_000_000)

  0.057521 seconds (973 allocations: 42.188 KiB)


3.14155996

Suppose we want to constrain range of values a vector's elements can take to be between -0.5 and 0.5

In [95]:
function squeeze_range!(x)
    n = length(x)
    for i = 1:n 
        if !(-0.5 ≤ x[i] ≤ 0.5) 
            x[i] = 0.5 * sign(x[i])
        end 
    end 
end


squeeze_range! (generic function with 1 method)

In [97]:
n = 100_000_000
v = randn(n)

@time squeeze_range!(v)

  0.507550 seconds (4 allocations: 160 bytes)


### Using `SharedArrays` for Parrallelism

In [98]:
#  Modify input value to be between -0.5 and 0.5
using SharedArrays

function shr_squeeze_range(x)
    n = length(x)
    @sync @distributed for i = 1:n 
        if !(-0.5 ≤ x[i] ≤ 0.5) 
            x[i] = 0.5 * sign(x[i])
        end 
    end 
end

shr_squeeze_range (generic function with 1 method)

In [100]:
n = 100_000_000
v = SharedArray(randn(n))

@time shr_squeeze_range(v);

  0.115931 seconds (1.65 k allocations: 79.719 KiB)


### Using Threads for Parallelism

In [101]:
# NOTE: JULIA_NUM_THREADS env variable is set in .bash_profile
using Base.Threads

function thr_squeeze_range!(x)
    n = length(x)
    @threads for i = 1:n 
        if !(-0.5 ≤ x[i] ≤ 0.5) 
            x[i] = 0.5 * sign(x[i])
        end 
    end 
end

thr_squeeze_range! (generic function with 1 method)

In [103]:
n = 100_000_000
v = randn(n)

@time thr_squeeze_range!(v)

  0.102906 seconds (5 allocations: 192 bytes)


# <center> Julia's Type System </center>

## Julia's Type System 
1. Dynamic
2. Parametric
3. Nominative

## Julia Types:
1. Every object has a type (e.g., `Int`, `Float64`, `Dict`, `Function`)
2. User-defined types behave just like base language types
3. Types (and functions) are parametric 
4. Type annotation can (optionally) be added and can improve performance

### Illustrating Multiple Dispatch

In [104]:
function foo(a::Int, b::Int) 
    println("I'm adding ints. Yay!!")
    res = a + b
    return res
end 

foo (generic function with 2 methods)

In [105]:
foo(3, 1)

I'm adding ints. Yay!!


4

In [106]:
# Adding method to foo()

function foo(a::Float64, b::Float64)
    println("Now I'm adding floats!")
    res = a + b
    return res 
end

foo (generic function with 2 methods)

In [107]:
foo(3.1, 2.8)

Now I'm adding floats!


5.9

### Using Concrete Types for Performance

In [108]:
function count_vals(v)
    n = length(v)
    cnts = Dict()
    for i = 1:n
        cnts[v[i]] = get(cnts,v[i], 0) + 1
    end
    return cnts
end

count_vals (generic function with 1 method)

In [110]:
x = rand(['a', 't', 'g', 'c'], 10_000_000)

@time res = count_vals(x)

  0.698905 seconds (20.00 M allocations: 305.114 MiB, 1.20% gc time)


Dict{Any,Any} with 4 entries:
  'g' => 2499542
  'a' => 2502378
  'c' => 2498021
  't' => 2500059

In [111]:
function count_vals2(v)
    n = length(v)
    cnts = Dict{Char, Int}()
    for i = 1:n
        cnts[v[i]] = get(cnts,v[i], 0) + 1
    end
    return cnts
end

count_vals2 (generic function with 2 methods)

In [112]:
x = rand(['a', 't', 'g', 'c'], 100_000_000)

@time res = count_vals2(x)

  1.227263 seconds (17.36 k allocations: 922.809 KiB)


Dict{Char,Int64} with 4 entries:
  'g' => 24998218
  'c' => 25003993
  'a' => 25002737
  't' => 24995052

In [113]:
function count_vals2(X::Matrix)
    n, p = size(X)
    cnts = Dict{Char, Int}()
    for j = 1:p 
        for i = 1:n
            cnts = get(cnts, X[i, j], 0) + 1
        end
    end
    return cnts 
end 

A = rand(['a', 't', 'g', 'c'], 1000, 1000)

count_vals(A) 

Dict{Any,Any} with 4 entries:
  'g' => 250218
  'c' => 249982
  'a' => 249481
  't' => 250319

## User-Defined Types 

1. User can define composite types (i.e., `struct`s)
2. A `struct` can be mutable or immutable
3. Immutable `struct`s are allocated on stack, mutable are on the heap

In [114]:
struct Dog
    name 
    age 
    words
end 


struct Cat
    name
    age
    words
end 


In [115]:

function say_name(x::Dog)
    s = "$(x.words), my name is $(x.name), and I'm a dog."
    println(s)
end

willie = Dog("William Lee", 11, "Woof")    

say_name(willie)

Woof, my name is William Lee, and I'm a dog.


In [116]:
function say_name(x::Cat)
    for i = 1:rand(1:10)
        print("$(x.words) ")
    end
end

ricky = Cat("Richard", 7, "Meow")

say_name(ricky)

Meow Meow Meow Meow Meow Meow Meow Meow Meow Meow 

In [119]:
# Example of multiple dispatch

function say_name(x::Dog, y::Cat)
    println("Two animals fight briefly...\n\nAnd the winner is: $(x.name)")
end 

say_name (generic function with 3 methods)

In [120]:
say_name(willy, ricky)

Two animals fight briefly...

And the winner is: William Lee


In [121]:
# Define new struct that is Dog/Cat hybrid

struct Dat 
    name
    age
    words
end 

### Operator Overloading

+ Can overload base language's operators (e.g., `+`, `*`, `/`)

In [122]:
import Base.+ 

function +(x::Dog, y::Cat)
    res = Dat(x.name, y.age, string(x.words, y.words))
    return res 
end


+ (generic function with 164 methods)

In [123]:
# Test our new method for +() function

willy + ricky

Dat("William Lee", 7, "WoofMeow")

# <center> Profiling and Introspection </center>

1. Profiling Tools
  * `@time` macro
  * BenchmarkTools.jl package
2. Introspection
  * Type hierarchy 
  * `typeof()`
  * `eltype()`
  * `@code_warntype` 
  * `@code_native`

In [ ]:
# Illustrating type hierarchy 

Int32 <: Number

In [ ]:
BigInt <: Real

In [ ]:
Bool <: Int64

In [ ]:
Float64 <: AbstractFloat

In [124]:
a = zeros(100)

typeof(a)

Array{Float64,1}

In [125]:
A = ones(100, 100)

typeof(A)

Array{Float64,2}

In [126]:
# Get type of elements in containers

b = randn(1000)

eltype(b)

Float64

In [127]:
# Get fields of a struct

fieldnames(Dog)

(:name, :age, :words)

In [ ]:
function count_vals(v)
    n = length(v)
    cnts = Dict()
    for i = 1:n
        cnts[v[i]] = get(cnts,v[i], 0) + 1
    end
    return cnts
end

x = rand(['a', 't', 'g', 'c'], 100_000)

@code_warntype count_vals(x)

In [ ]:
z = rand(['a', 't', 'g', 'c'], 100_000)

@code_warntype count_vals2(z)

# <center>Integration with other Languages</center>

## Julia and other Languages

Julia has zero-cost interfaces with many technical computing languages:  
1. PyCall.jl
2. RCall.jl
3. `ccall()`
4. Cxx.jl
5. Matlab.jl


### <center>Examples using RCall.jl and Cxx.jl</center>

# <center> Tooling and Package Ecosystem </center>

1. Package manager
  * GitHub
  * Packages installed directly from Julia session using `Pkg.add("<PACKAGE-NAME>")` 
2. Editors and IDEs
  * Atom (Juno plugin)
  * VS Code
  * Emacs (ESS plugin)
  * Jupyter
  * Jupyter Lab

# <center> Challenges </center>

1. Julia is still young
  * Package ecosystem is still growing
  * StackOverflow answers are a bit sparse
  * Breaking changes still occur (mostly in packages, not base language)